In [68]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [35]:
DATA_PATH = 'march-machine-learning-mania-2023/'

## Data Preparation

### Seeds

In [36]:
df_seeds = pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv")
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


### Season results

Add total rebounds for each team, and for the whole game (to later calculate rebound percentages)

In [37]:
df_season_results = pd.read_csv(DATA_PATH + "MRegularSeasonDetailedResults.csv")

df_season_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

In [38]:
df_season_results['WTR'] = df_season_results['WOR'] + df_season_results['WDR']
df_season_results['LTR'] = df_season_results['LOR'] + df_season_results['LDR']
df_season_results['TR'] = df_season_results['WTR'] + df_season_results['LTR']
df_season_results['ScoreGap'] = df_season_results['WScore'] - df_season_results['LScore']

df_season_results.drop(['WStl', 'WBlk', 'WPF', 'LStl', 'LBlk', 'LPF'], axis=1, inplace=True)

### Features
Aggregate the results for the fields we will later use for our features.

For each team at each season, computing:
- Number of wins
- Number of losses
- Average score gap of wins
- Average score gap of losses
- Total field goals made and attempted for wins and losses
- Total 3-point field goals made and attempted for wins and losses
- Total free throws made and attempted for wins and losses
- Total rebounds by the team for wins and losses
- Total rebounds in the game for wins and losses
- Total assists for wins and losses
- Total turnovers for wins and losses

And use the following features ([explained here](https://en.wikipedia.org/wiki/Basketball_statistics)):

- Win Ratio
- Average score gap
- Field goal %
- 3-point field goal %
- Free throw %
- [Effective field goal %](https://en.wikipedia.org/wiki/Effective_field_goal_percentage)
- [True shooting %](https://en.wikipedia.org/wiki/True_shooting_percentage)
- Total rebound %
- Assist to turnover ratio
- Turnover %
- Offensive rating (Points scored per possession)
- Defensive rating (Points allowed per opposition possession)
- [Net rating (Offensive rating - Defensive rating)](https://www.pivotanalysis.com/post/net-rating)


In [39]:
num_win = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})
num_loss = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"})
gap_win = df_season_results.groupby(['Season', 'WTeamID']).mean().reset_index()[['Season', 'WTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapWins", "WTeamID": "TeamID"})
gap_loss = df_season_results.groupby(['Season', 'LTeamID']).mean().reset_index()[['Season', 'LTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapLosses", "LTeamID": "TeamID"})
fgm_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFGM']].rename(columns={"WFGM": "FGMWins", "WTeamID": "TeamID"})
fgm_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFGM']].rename(columns={"LFGM": "FGMLosses", "LTeamID": "TeamID"})
fga_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFGA']].rename(columns={"WFGA": "FGAWins", "WTeamID": "TeamID"})
fga_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFGA']].rename(columns={"LFGA": "FGALosses", "LTeamID": "TeamID"})
fgm3_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFGM3']].rename(columns={"WFGM3": "FGM3Wins", "WTeamID": "TeamID"})
fgm3_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFGM3']].rename(columns={"LFGM3": "FGM3Losses", "LTeamID": "TeamID"})
fga3_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFGA3']].rename(columns={"WFGA3": "FGA3Wins", "WTeamID": "TeamID"})
fga3_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFGA3']].rename(columns={"LFGA3": "FGA3Losses", "LTeamID": "TeamID"})
ftm_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFTM']].rename(columns={"WFTM": "FTMWins", "WTeamID": "TeamID"})
ftm_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFTM']].rename(columns={"LFTM": "FTMLosses", "LTeamID": "TeamID"})
fta_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WFTA']].rename(columns={"WFTA": "FTAWins", "WTeamID": "TeamID"})
fta_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LFTA']].rename(columns={"LFTA": "FTALosses", "LTeamID": "TeamID"})
teamreb_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WTR']].rename(columns={"WTR": "TeamRebWins", "WTeamID": "TeamID"})
teamreb_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LTR']].rename(columns={"LTR": "TeamRebLosses", "LTeamID": "TeamID"})
gamereb_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'TR']].rename(columns={"TR": "GameRebWins", "WTeamID": "TeamID"})
gamereb_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'TR']].rename(columns={"TR": "GameRebLosses", "LTeamID": "TeamID"})
ast_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WAst']].rename(columns={"WAst": "AstWins", "WTeamID": "TeamID"})
ast_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LAst']].rename(columns={"LAst": "AstLosses", "LTeamID": "TeamID"})
to_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'WTO']].rename(columns={"WTO": "TOWins", "WTeamID": "TeamID"})
to_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'LTO']].rename(columns={"LTO": "TOLosses", "LTeamID": "TeamID"})
opp_pts_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'LScore']].rename(columns={"LScore": "oppPTSWins", "WTeamID": "TeamID"})
opp_pts_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'WScore']].rename(columns={"WScore": "oppPTSLosses", "LTeamID": "TeamID"})
opp_fga_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'LFGA']].rename(columns={"LFGA": "oppFGAWins", "WTeamID": "TeamID"})
opp_fga_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'WFGA']].rename(columns={"WFGA": "oppFGALosses", "LTeamID": "TeamID"})
opp_fta_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'LFTA']].rename(columns={"LFTA": "oppFTAWins", "WTeamID": "TeamID"})
opp_fta_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'WFTA']].rename(columns={"WFTA": "oppFTALosses", "LTeamID": "TeamID"})
opp_to_win = df_season_results.groupby(['Season', 'WTeamID']).sum().reset_index()[['Season', 'WTeamID', 'LTO']].rename(columns={"LTO": "oppTOWins", "WTeamID": "TeamID"})
opp_to_loss = df_season_results.groupby(['Season', 'LTeamID']).sum().reset_index()[['Season', 'LTeamID', 'WTO']].rename(columns={"WTO": "oppTOLosses", "LTeamID": "TeamID"})


In [40]:
df_features_season_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})
df_features_season = pd.concat([
  df_features_season_w, 
  df_features_season_l
], axis=0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

#### Merge, compute aggregates

In [41]:
df_features_season = df_features_season.merge(num_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(num_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fgm_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fgm_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fga_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fga_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fgm3_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fgm3_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fga3_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fga3_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(ftm_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(ftm_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fta_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(fta_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(teamreb_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(teamreb_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gamereb_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gamereb_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(ast_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(ast_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(to_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(to_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_pts_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_pts_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_fga_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_fga_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_fta_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_fta_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_to_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(opp_to_loss, on=['Season', 'TeamID'], how='left')

df_features_season.fillna(0, inplace=True)

df_features_season['FGM'] = df_features_season['FGMWins'] + df_features_season['FGMLosses']
df_features_season['FGA'] = df_features_season['FGAWins'] + df_features_season['FGALosses']
df_features_season['FGM3'] = df_features_season['FGM3Wins'] + df_features_season['FGM3Losses']
df_features_season['FGA3'] = df_features_season['FGA3Wins'] + df_features_season['FGA3Losses']
df_features_season['FTM'] = df_features_season['FTMWins'] + df_features_season['FTMLosses']
df_features_season['FTA'] = df_features_season['FTAWins'] + df_features_season['FTALosses']
df_features_season['TeamReb'] = df_features_season['TeamRebWins'] + df_features_season['TeamRebLosses']
df_features_season['GameReb'] = df_features_season['GameRebWins'] + df_features_season['GameRebLosses']
df_features_season['Ast'] = df_features_season['AstWins'] + df_features_season['AstLosses']
df_features_season['TO'] = df_features_season['TOWins'] + df_features_season['TOLosses']
df_features_season['PTS'] = 2 * df_features_season['FGM'] + df_features_season['FGM3'] + df_features_season['FTM']
df_features_season['POSS'] = df_features_season['FGA'] + 0.44 * df_features_season['FTA'] + df_features_season['TO']
df_features_season['oppPTS'] = df_features_season['oppPTSWins'] + df_features_season['oppPTSLosses']
df_features_season['oppFGA'] = df_features_season['oppFGAWins'] + df_features_season['oppFGALosses']
df_features_season['oppFTA'] = df_features_season['oppFTAWins'] + df_features_season['oppFTALosses']
df_features_season['oppTO'] = df_features_season['oppTOWins'] + df_features_season['oppTOLosses']
df_features_season['oppPOSS'] = df_features_season['oppFGA'] + 0.44 * df_features_season['oppFTA'] + df_features_season['oppTO']

#### Compute features

In [42]:
df_features_season['WinRatio'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])
df_features_season['GapAvg'] = (
    (df_features_season['NumWins'] * df_features_season['GapWins'] - 
    df_features_season['NumLosses'] * df_features_season['GapLosses'])
    / (df_features_season['NumWins'] + df_features_season['NumLosses'])
)
df_features_season['FieldGoal%'] = df_features_season['FGM'] / df_features_season['FGA']
df_features_season['3PointFG%'] = df_features_season['FGM3'] / df_features_season['FGA3']
df_features_season['FreeThrow%'] = df_features_season['FTM'] / df_features_season['FTA']
df_features_season['effectiveFG%'] = (df_features_season['FGM'] + 0.5 * df_features_season['FGM3']) / df_features_season['FGA']
df_features_season['TrueShooting%'] = df_features_season['PTS'] / (2 * df_features_season['FGA'] + 0.88 * df_features_season['FTA'])
df_features_season['TotalRebound%'] = df_features_season['TeamReb'] / df_features_season['GameReb']
df_features_season['AssistToTurnover'] = df_features_season['Ast'] / df_features_season['TO']
df_features_season['Turnover%'] = df_features_season['TO'] / df_features_season['POSS']
df_features_season['OffRating'] = df_features_season['PTS'] / df_features_season['POSS'] * 100
df_features_season['DefRating'] = df_features_season['oppPTS'] / df_features_season['oppPOSS'] * 100
df_features_season['NetRating'] = df_features_season['OffRating'] - df_features_season['DefRating']

# features = ['FieldGoal%', '3PointFG%', 'FreeThrow%', 'effectiveFG%', 'TrueShooting%', 'TotalRebound%', 'AssistToTurnover', 'Turnover%', 'OffRating', 'DefRating', 'NetRating']
features = ['TrueShooting%', 'TotalRebound%', 'AssistToTurnover', 'OffRating', 'DefRating', 'NetRating']
targets = ['WinRatio', 'GapAvg']

keep = ['Season', 'TeamID'] + features + targets
df_features_season.drop([col for col in df_features_season.columns.values if col not in keep], axis=1, inplace=True)
df_features_season.head()

,Season,TeamID,WinRatio,GapAvg,TrueShooting%,TotalRebound%,AssistToTurnover,OffRating,DefRating,NetRating
0,2003,1102,0.428571,0.250000,0.605015,0.413793,1.137500,97.461028,89.253758,8.207270
1,2003,1103,0.481481,0.629630,0.585912,0.465738,1.205279,98.649426,95.211451,3.437975
2,2003,1104,0.607143,4.285714,0.521831,0.527903,0.911290,86.962758,84.525358,2.437401
3,2003,1105,0.269231,-4.884615,0.503801,0.480565,0.779381,79.848690,86.566360,-6.717671
4,2003,1106,0.464286,-0.142857,0.508613,0.517647,0.685535,79.942904,81.596270,-1.653366


#### Correlations

In [43]:
def correlations(df, features, target, extra = ''):
  corrs = {}
  for feature in features + [extra]:
    if feature in df.columns:
      corrs[feature] = round(df[feature].corr(df[target]), 4)
  return dict(sorted(corrs.items(), key=lambda item: abs(item[1]), reverse=True))

Correlations with WinRatio

In [44]:
correlations(df_features_season, features, 'WinRatio')

{'NetRating': 0.9225,
 'OffRating': 0.7018,
 'TrueShooting%': 0.6557,
 'DefRating': -0.6537,
 'TotalRebound%': 0.6035,
 'AssistToTurnover': 0.5982}

Correlation with GapAvg (aka avg +/-)

In [45]:
correlations(df_features_season, features, 'GapAvg')

{'NetRating': 0.9635,
 'OffRating': 0.7347,
 'TrueShooting%': 0.6807,
 'DefRating': -0.6807,
 'TotalRebound%': 0.6486,
 'AssistToTurnover': 0.6479}

### Tourney Results 

In [46]:
df_tourney_results = pd.read_csv(DATA_PATH + "MNCAATourneyCompactResults.csv")

df_tourney_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

## Feature Engineering

### Train Data

In [47]:
df = df_tourney_results.copy()
df = df[df['Season'] >= 2016].reset_index(drop=True)
# df.head()

#### Seeds

In [48]:
df = pd.merge(
    df, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'WTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedW'})

df = pd.merge(
    df, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'LTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedL'})

In [49]:
def treat_seed(seed):
    return int(re.sub("[^0-9]", "", seed))

In [50]:
df['SeedW'] = df['SeedW'].apply(treat_seed)
df['SeedL'] = df['SeedL'].apply(treat_seed)
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL
0,2016,134,1195,96,1192,65,16,16
1,2016,134,1455,70,1435,50,11,11
2,2016,135,1221,59,1380,55,16,16
3,2016,135,1276,67,1409,62,11,11
4,2016,136,1114,85,1345,83,12,5


#### Season Stats

In [51]:
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'WinRatio': 'WinRatioW',
    'GapAvg': 'GapAvgW',
    'TrueShooting%' : 'TrueShootingW',
    'TotalRebound%' : 'TotalReboundW',
    'AssistToTurnover' : 'AssistToTurnoverW',
    'OffRating' : 'OffRatingW',
    'DefRating' : 'DefRatingW',
    'NetRating' : 'NetRatingW',
}).drop(columns='TeamID', axis=1)
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'WinRatio': 'WinRatioL',
    'GapAvg': 'GapAvgL',
    'TrueShooting%' : 'TrueShootingL',
    'TotalRebound%' : 'TotalReboundL',
    'AssistToTurnover' : 'AssistToTurnoverL',
    'OffRating' : 'OffRatingL',
    'DefRating' : 'DefRatingL',
    'NetRating' : 'NetRatingL',
}).drop(columns='TeamID', axis=1)
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL,WinRatioW,GapAvgW,...,DefRatingW,NetRatingW,WinRatioL,GapAvgL,TrueShootingL,TotalReboundL,AssistToTurnoverL,OffRatingL,DefRatingL,NetRatingL
0,2016,134,1195,96,1192,65,16,16,0.551724,2.724138,...,89.733136,2.796919,0.548387,-1.677419,0.551992,0.461159,1.084367,93.023619,93.011525,0.012094
1,2016,134,1455,70,1435,50,11,11,0.741935,13.709677,...,80.251647,14.255515,0.593750,9.406250,0.572964,0.508656,1.278075,97.567177,83.204633,14.362544
2,2016,135,1221,59,1380,55,16,16,0.424242,-4.333333,...,93.839799,-5.128661,0.612903,1.870968,0.527352,0.484554,0.911681,90.325959,86.226258,4.099701
3,2016,135,1276,67,1409,62,11,11,0.636364,6.242424,...,91.844232,9.245695,0.645161,4.322581,0.543994,0.486559,1.268657,93.882495,87.282602,6.599894
4,2016,136,1114,85,1345,83,12,5,0.870968,9.935484,...,81.264621,15.238154,0.764706,13.147059,0.578135,0.574266,1.472906,98.187087,84.600804,13.586282


#### Add Symmetrical

In [52]:
def add_losing_matches(df):
  win_rename = {
    "WTeamID": "TeamIDA", 
    "WScore" : "ScoreA", 
    "LTeamID" : "TeamIDB",
    "LScore": "ScoreB",
    }
  win_rename.update({c : c[:-1] + "A" for c in df.columns if c.endswith('W')})
  win_rename.update({c : c[:-1] + "B" for c in df.columns if c.endswith('L')})

  lose_rename = {
    "WTeamID": "TeamIDB", 
    "WScore" : "ScoreB", 
    "LTeamID" : "TeamIDA",
    "LScore": "ScoreA",
  }
  lose_rename.update({c : c[:-1] + "B" for c in df.columns if c.endswith('W')})
  lose_rename.update({c : c[:-1] + "A" for c in df.columns if c.endswith('L')})

  win_df = df.copy()
  lose_df = df.copy()

  win_df = win_df.rename(columns=win_rename)
  lose_df = lose_df.rename(columns=lose_rename)

  return pd.concat([win_df, lose_df], axis=0, sort=False)
df = add_losing_matches(df)

#### Add Differences

In [53]:
cols_to_diff = [
    'Seed', 'WinRatio', 'GapAvg', 'TrueShooting', 'TotalRebound', 'AssistToTurnover', 'OffRating', 'DefRating', 'NetRating'
]
for col in cols_to_diff:
    df[col + 'Diff'] = df[col + 'A'] - df[col + 'B']
df.head()

,Season,DayNum,TeamIDA,ScoreA,TeamIDB,ScoreB,SeedA,SeedB,WinRatioA,GapAvgA,...,NetRatingB,SeedDiff,WinRatioDiff,GapAvgDiff,TrueShootingDiff,TotalReboundDiff,AssistToTurnoverDiff,OffRatingDiff,DefRatingDiff,NetRatingDiff
0,2016,134,1195,96,1192,65,16,16,0.551724,2.724138,...,0.012094,0,0.003337,4.401557,-0.009675,0.065327,-0.009655,-0.493564,-3.278389,2.784825
1,2016,134,1455,70,1435,50,11,11,0.741935,13.709677,...,14.362544,0,0.148185,4.303427,-0.031594,0.023883,0.177663,-3.060015,-2.952986,-0.107028
2,2016,135,1221,59,1380,55,16,16,0.424242,-4.333333,...,4.099701,0,-0.188661,-6.204301,-0.006166,-0.026701,0.356275,-1.614821,7.613541,-9.228363
3,2016,135,1276,67,1409,62,11,11,0.636364,6.242424,...,6.599894,0,-0.008798,1.919844,0.038405,0.005756,0.270284,7.207432,4.561631,2.645801
4,2016,136,1114,85,1345,83,12,5,0.870968,9.935484,...,13.586282,7,0.106262,-3.211575,-0.016436,-0.082699,-0.211077,-1.684312,-3.336184,1.651872


### Test Data

#### Preparing

In [54]:
df_test = pd.read_csv(DATA_PATH + "SampleSubmission2023.csv")
df_test['Season'] = df_test['ID'].apply(lambda x: int(x.split('_')[0]))
df_test['TeamIDA'] = df_test['ID'].apply(lambda x: int(x.split('_')[1]))
df_test['TeamIDB'] = df_test['ID'].apply(lambda x: int(x.split('_')[2]))
df_test.head()

,ID,Pred,Season,TeamIDA,TeamIDB
0,2023_1101_1102,0.5,2023,1101,1102
1,2023_1101_1103,0.5,2023,1101,1103
2,2023_1101_1104,0.5,2023,1101,1104
3,2023_1101_1105,0.5,2023,1101,1105
4,2023_1101_1106,0.5,2023,1101,1106


#### Seeds

In [55]:
df_test = pd.merge(
  df_test,
  df_seeds,
  how='left',
  left_on=['Season', 'TeamIDA'],
  right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedA'}).fillna('W00')
df_test = pd.merge(
  df_test, 
  df_seeds, 
  how='left', 
  left_on=['Season', 'TeamIDB'], 
  right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedB'}).fillna('W00')
df_test['SeedA'] = df_test['SeedA'].apply(treat_seed)
df_test['SeedB'] = df_test['SeedB'].apply(treat_seed)
df_test.head()

,ID,Pred,Season,TeamIDA,TeamIDB,SeedA,SeedB
0,2023_1101_1102,0.5,2023,1101,1102,0,0
1,2023_1101_1103,0.5,2023,1101,1103,0,0
2,2023_1101_1104,0.5,2023,1101,1104,0,1
3,2023_1101_1105,0.5,2023,1101,1105,0,0
4,2023_1101_1106,0.5,2023,1101,1106,0,0


#### Season Stats

In [56]:
df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIDA'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'WinRatio': 'WinRatioA',
    'GapAvg': 'GapAvgA',
    'TrueShooting%' : 'TrueShootingA',
    'TotalRebound%' : 'TotalReboundA',
    'AssistToTurnover' : 'AssistToTurnoverA',
    'OffRating' : 'OffRatingA',
    'DefRating' : 'DefRatingA',
    'NetRating' : 'NetRatingA',
}).drop(columns='TeamID', axis=1)

df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIDB'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'WinRatio': 'WinRatioB',
    'GapAvg': 'GapAvgB',
    'TrueShooting%' : 'TrueShootingB',
    'TotalRebound%' : 'TotalReboundB',
    'AssistToTurnover' : 'AssistToTurnoverB',
    'OffRating' : 'OffRatingB',
    'DefRating' : 'DefRatingB',
    'NetRating' : 'NetRatingB',
}).drop(columns='TeamID', axis=1)

In [57]:
def average_if_na(df, features):
  for feature in features:
    df[feature] = df[feature].fillna(df[feature].mean())

In [58]:
not_feature = ['ID', 'Pred', 'Season', 'TeamIDA', 'TeamIDB']
features = [col for col in df_test.columns.values if col not in not_feature]
average_if_na(df_test, features)


#### Differences

In [59]:
for col in cols_to_diff:
    df_test[col + 'Diff'] = df_test[col + 'A'] - df_test[col + 'B']

### Target

In [60]:
df['ScoreDiff'] = df['ScoreA'] - df["ScoreB"]
df['WinA'] = (df['ScoreDiff'] > 0).astype(int)

## Feature selection

In [61]:
def variants(features):
  res = []
  for feature in features:
    res.append(feature + "A")
    res.append(feature + "B")
    res.append(feature + "Diff")
  return res

In [62]:
not_feature = ['Season', 'DayNum', 'TeamIDA', 'ScoreA', 'TeamIDB', 'ScoreB', 'ScoreDiff', 'WinA']
actual_features = ['Seed', 'WinRatio', 'GapAvg', 'TrueShooting', 'TotalRebound', 'AssistToTurnover', 'OffRating', 'DefRating', 'NetRating']
also_not = variants([f for f in actual_features if f not in keep])
features = [col for col in df.columns.values if col not in not_feature + also_not]
features

['WinRatioA',
 'GapAvgA',
 'AssistToTurnoverA',
 'OffRatingA',
 'DefRatingA',
 'NetRatingA',
 'WinRatioB',
 'GapAvgB',
 'AssistToTurnoverB',
 'OffRatingB',
 'DefRatingB',
 'NetRatingB',
 'WinRatioDiff',
 'GapAvgDiff',
 'AssistToTurnoverDiff',
 'OffRatingDiff',
 'DefRatingDiff',
 'NetRatingDiff']

## Modelling

In [63]:
def rescale(features, df_train, df_val, df_test=None):
  min_ = df_train[features].min()
  max_ = df_train[features].max()
  
  df_train[features] = (df_train[features] - min_) / (max_ - min_)
  df_val[features] = (df_val[features] - min_) / (max_ - min_)
  
  if df_test is not None:
      df_test[features] = (df_test[features] - min_) / (max_ - min_)
      
  return df_train, df_val, df_test

### Cross Validation

In [96]:
def kfold(df, df_test_=None, features=features, plot=False, verbose=0, mode="reg"):
  seasons = df['Season'].unique()
  cvs = []
  pred_tests = []
  target = "ScoreDiff" if mode == "reg" else "WinA"
  
  for season in seasons[1:]:
    if verbose:
      print(f'\nValidating on season {season}')
    
    df_train = df[df['Season'] < season].reset_index(drop=True).copy()
    df_val = df[df['Season'] == season].reset_index(drop=True).copy()
    df_test = df_test_.copy()
    
    df_train, df_val, df_test = rescale(features, df_train, df_val, df_test)
    
    if mode == "reg":
      model = ElasticNet(alpha=1, l1_ratio=0.5) # model = LinearRegression()
    elif mode == "nb":
      model = GaussianNB()
    else:
      model = LogisticRegression(C=1)

    model.fit(df_train[features], df_train[target])
    
    if mode == "reg":
        pred = model.predict(df_val[features])
    else:
        pred = model.predict_proba(df_val[features])[:, 1]
    
    if df_test is not None:
      if mode == "reg":
        pred_test = model.predict(df_test[features]) # print( pred_test.max() - pred_test.min())
        pred_test = (pred_test - pred_test.min()) / (pred_test.max() - pred_test.min())
      else:
        pred_test = model.predict(df_test[features])
        print(f"Accuracy: {model.score(df_test[features], pred_test)}")
          
      pred_tests.append(pred_test)
        
    # if plot:
    #   plt.figure(figsize=(15, 6))
    #   plt.subplot(1, 2, 1)
    #   plt.scatter(pred, df_val['ScoreDiff'].values, s=5)
    #   plt.title('Prediction vs Score Diff')
    #   plt.grid(True)
    #   plt.subplot(1, 2, 2)
    #   sns.histplot(pred, bins=20)
    #   plt.title('Predictions probability repartition')
    #   plt.show()
    pred = (pred - pred.min()) / (pred.max() - pred.min())
    pred = np.clip(pred, 0, 1)

    score = ((df_val['WinA'].values - pred) ** 2).mean()
    cvs.append(score)

    if verbose:
      print(f'\t -> Scored {score:.3f}')
  print(f'\n Local CV is {np.mean(cvs):.3f}')
  # Calculate accuracy
  
  return pred_tests

## Results

In [97]:
pred_tests = kfold(df, df_test, plot=False, verbose=1, mode="cls")


Validating on season 2017
Accuracy: 1.0
	 -> Scored 0.253

Validating on season 2018
Accuracy: 1.0
	 -> Scored 0.222

Validating on season 2019
Accuracy: 1.0
	 -> Scored 0.203

Validating on season 2021
Accuracy: 1.0
	 -> Scored 0.239

Validating on season 2022
Accuracy: 1.0
	 -> Scored 0.258

 Local CV is 0.235


In [98]:
pred_tests = kfold(df, df_test, plot=False, verbose=1, mode="nb")


Validating on season 2017
Accuracy: 1.0
	 -> Scored 0.319

Validating on season 2018
Accuracy: 1.0
	 -> Scored 0.312

Validating on season 2019
Accuracy: 1.0
	 -> Scored 0.312

Validating on season 2021
Accuracy: 1.0
	 -> Scored 0.339

Validating on season 2022
Accuracy: 1.0
	 -> Scored 0.365

 Local CV is 0.329
